## 1、代码部署

In [25]:
import os
from argparse import Namespace
from collections import Counter
import json
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None, mask_token="<MASK>", add_unk=True, unk_token="<UNK>"):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
            mask_token (str): the MASK token to add into the Vocabulary; indicates
                a position that will not be used in updating the model's parameters
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary

        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token
                              for token, idx in self._token_to_idx.items()}

        self._add_unk = add_unk
        self._unk_token = unk_token
        self._mask_token = mask_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token)

    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token,
                'mask_token': self._mask_token}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index

    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary

        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token
          or the UNK index if token isn't present.

        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
              for the UNK functionality
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index

        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)


class CBOWVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""

    def __init__(self, cbow_vocab):
        """
        Args:
            cbow_vocab (Vocabulary): maps words to integers
        """
        self.cbow_vocab = cbow_vocab

    def vectorize(self, context):
        """
        Args:
            context (str): the string of words separated by a space
        """
        indices = [self.cbow_vocab.lookup_token(token) for token in context.split(' ')]
        return np.array(indices)

    @classmethod
    def from_dataframe(cls, cbow_df):
        """Instantiate the vectorizer from the dataset dataframe

        Args:
            cbow_df (pandas.DataFrame): the target dataset
        Returns:
            an instance of the CBOWVectorizer
        """
        cbow_vocab = Vocabulary()

        # 添加您希望确保存在的词
        required_tokens = ['monster', 'frankenstein', 'science', 'sickness', 'lonely', 'happy']
        for token in required_tokens:
            cbow_vocab.add_token(token)

        for index, row in cbow_df.iterrows():
            for token in row.surname.split(' '):
                cbow_vocab.add_token(token)
            cbow_vocab.add_token(row.nationality)

        return cls(cbow_vocab)

    @classmethod
    def from_serializable(cls, contents):
        cbow_vocab = \
            Vocabulary.from_serializable(contents['cbow_vocab'])
        return cls(cbow_vocab=cbow_vocab)

    def to_serializable(self):
        return {'cbow_vocab': self.cbow_vocab.to_serializable()}


class CBOWDataset(Dataset):
    def __init__(self, cbow_df, vectorizer):
        """
        Args:
            cbow_df (pandas.DataFrame): the dataset
            vectorizer (CBOWVectorizer): vectorizer instatiated from dataset
        """
        self.cbow_df = cbow_df
        self._vectorizer = vectorizer

        measure_len = lambda surname: len(surname.split(" "))
        self._max_seq_length = max(map(measure_len, cbow_df.surname))

        self.train_df = self.cbow_df[self.cbow_df.split == 'train']
        self.train_size = len(self.train_df)

        self.val_df = self.cbow_df[self.cbow_df.split == 'val']
        self.validation_size = len(self.val_df)

        self.test_df = self.cbow_df[self.cbow_df.split == 'test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vectorizer(cls, cbow_csv):
        """Load dataset and make a new vectorizer from scratch

        Args:
            cbow_csv (str): location of the dataset
        Returns:
            an instance of CBOWDataset
        """
        cbow_df = pd.read_csv(cbow_csv)
        train_cbow_df = cbow_df[cbow_df.split == 'train']
        return cls(cbow_df, CBOWVectorizer.from_dataframe(train_cbow_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, cbow_csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer.
        Used in the case in the vectorizer has been cached for re-use

        Args:
            cbow_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of CBOWDataset
        """
        cbow_df = pd.read_csv(cbow_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(cbow_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file

        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of CBOWVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return CBOWVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json

        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets

        Args:
            index (int): the index to the data point
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]

        context_vector = \
            self._vectorizer.vectorize(row.surname)
        target_index = self._vectorizer.cbow_vocab.lookup_token(row.nationality)

        return {'x_data': context_vector,
                'y_target': target_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset

        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size


def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict


class CBOWClassifier(nn.Module):  # Simplified cbow Model
    def __init__(self, vocabulary_size, embedding_size, padding_idx=0):
        """
        Args:
            vocabulary_size (int): number of vocabulary items, controls the
                number of embeddings and prediction vector size
            embedding_size (int): size of the embeddings
            padding_idx (int): default 0; Embedding will not use this index
        """
        super(CBOWClassifier, self).__init__()

        self.embedding = nn.Embedding(num_embeddings=vocabulary_size,
                                      embedding_dim=embedding_size,
                                      padding_idx=padding_idx)
        self.fc1 = nn.Linear(in_features=embedding_size,
                             out_features=vocabulary_size)

    def forward(self, x_in, apply_softmax=False):
        """The forward pass of the classifier

        Args:
            x_in (torch.Tensor): an input data tensor.
                x_in.shape should be (batch, input_dim)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, output_dim)
        """
        # 嵌入层
        x_embedded = self.embedding(x_in)
        # 嵌入后求和
        x_embedded_sum = F.dropout(x_embedded.sum(dim=1), 0.3)
        # 全连接层
        y_out = self.fc1(x_embedded_sum)

        # 调试信息
        print(f"输入序列形状：{x_in.shape} -> 嵌入后：{x_embedded.shape}")
        print(f"嵌入求和后：{x_embedded_sum.shape} -> 全连接后：{y_out.shape}")

        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)

        return y_out


def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}


def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state


def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100


def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)


def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)


args = Namespace(
    # Data and Path information
    cbow_csv="surnames_with_splits.csv",  # 修改为正确的文件名
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/ch5/cbow",
    # Model hyper parameters
    embedding_size=50,
    # Training hyper parameters
    seed=1337,
    num_epochs=5,
    learning_rate=0.0001,
    batch_size=32,
    early_stopping_criteria=5,
    # Runtime options
    cuda=True,
    catch_keyboard_interrupt=True,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)

    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")

print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)

if args.reload_from_files:
    print("Loading dataset and loading vectorizer")
    dataset = CBOWDataset.load_dataset_and_load_vectorizer(args.cbow_csv,
                                                           args.vectorizer_file)
else:
    print("Loading dataset and creating vectorizer")
    dataset = CBOWDataset.load_dataset_and_make_vectorizer(args.cbow_csv)
    dataset.save_vectorizer(args.vectorizer_file)

vectorizer = dataset.get_vectorizer()

# 数据预处理验证
print("Vocabulary特殊标记索引：", 
      f"<MASK>:{vectorizer.cbow_vocab.mask_index}",
      f"<UNK>:{vectorizer.cbow_vocab.unk_index}")

# 打印样本向量化结果
sample_str = "Zhang"
vec = vectorizer.vectorize(sample_str)
print(f"样本'{sample_str}'的向量化结果:\n{vec}\n有效长度:{len(sample_str.split())}")

classifier = CBOWClassifier(vocabulary_size=len(vectorizer.cbow_vocab),
                            embedding_size=args.embedding_size)

classifier = classifier.to(args.device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)
train_state = make_train_state(args)

epoch_bar = tqdm(desc='training routine',
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size),
                          position=1,
                          leave=True)
dataset.set_split('val')
val_bar = tqdm(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size),
                        position=1,
                        leave=True)

try:
    for epoch_index in range(args.num_epochs):
        print(f"\nEpoch [{epoch_index + 1}/{args.num_epochs}]")
        print("-" * 50)
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on

        dataset.set_split('train')
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):

            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(x_in=batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            if batch_index % 500 == 0:
                train_bar.set_postfix(loss=running_loss, acc=0.25, epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            # compute the output
            y_pred = classifier(x_in=batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            if batch_index % 500 == 0:
                val_bar.set_postfix(loss=running_loss, acc=0.25, epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        if train_state['stop_early']:
            break

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
except KeyboardInterrupt:
    print("Exiting loop")

# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))
classifier = classifier.to(args.device)
loss_func = nn.CrossEntropyLoss()

dataset.set_split('test')
batch_generator = generate_batches(dataset,
                                   batch_size=args.batch_size,
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred = classifier(x_in=batch_dict['x_data'])

    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))


def pretty_print(results):
    """
    Pretty print embedding results.
    """
    for item in results:
        print("...[%.2f] - %s" % (item[1], item[0]))


def get_closest(target_word, word_to_idx, embeddings, n=5):
    """
    Get the n closest words to your word.
    """
    if target_word.lower() not in word_to_idx:
        print(f"Word '{target_word}' not found in vocabulary.")
        return []

    word_embedding = embeddings[word_to_idx[target_word.lower()]]
    distances = []
    for word, index in word_to_idx.items():
        if word == "<MASK>" or word == target_word:
            continue
        distances.append((word, torch.dist(word_embedding, embeddings[index])))

    results = sorted(distances, key=lambda x: x[1])[1:n + 2]
    return results


word = 'monster'
embeddings = classifier.embedding.weight.data
word_to_idx = vectorizer.cbow_vocab._token_to_idx

# 检查目标词是否在词汇表中
if word.lower() in word_to_idx:
    pretty_print(get_closest(word, word_to_idx, embeddings, n=5))
else:
    print(f"The word '{word}' is not in the vocabulary.")

target_words = ['frankenstein', 'monster', 'science', 'sickness', 'lonely', 'happy']

for target_word in target_words:
    print(f"======={target_word}=======")
    if target_word.lower() not in word_to_idx:
        print(f"The word '{target_word}' is not in the vocabulary.")
        continue
    pretty_print(get_closest(target_word, word_to_idx, embeddings, n=5))

Expanded filepaths: 
	model_storage/ch5/cbow\vectorizer.json
	model_storage/ch5/cbow\model.pth
Using CUDA: False
Loading dataset and creating vectorizer
Vocabulary特殊标记索引： <MASK>:0 <UNK>:1
样本'Zhang'的向量化结果:
[148]
有效长度:1


training routine: 100%|██████████████████████████████████████████████████████████████████| 5/5 [00:34<00:00,  6.91s/it]

split=train:   0%|                                               | 0/240 [00:34<?, ?it/s, acc=0.25, epoch=4, loss=8.77]

split=val:   0%|                                                  | 0/51 [00:34<?, ?it/s, acc=0.25, epoch=4, loss=8.05]

split=train:   7%|██▌                                  | 17/240 [00:00<00:01, 161.23it/s, acc=0.25, epoch=0, loss=8.75]


Epoch [1/5]
--------------------------------------------------
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：t

输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])


split=train:  22%|████████▏                            | 53/240 [00:00<00:01, 170.19it/s, acc=0.25, epoch=0, loss=8.75]

...


split=val:   0%|                                          | 0/51 [00:08<00:01, 33.58it/s, acc=0.25, epoch=4, loss=8.05]

输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Si

training routine: 100%|██████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.75s/it]

输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Size([32, 50]) -> 全连接后：torch.Size([32, 6466])
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
嵌入求和后：torch.Si


split=train:   0%|                                      | 0/240 [00:12<00:01, 170.77it/s, acc=0.25, epoch=4, loss=8.77]

## 数据预处理验证

In [27]:
# 数据预处理验证
# 在 SurnameVectorizer 初始化后添加
print("Char Vocab特殊标记索引：", 
      f"<MASK>:{vectorizer.cbow_vocab.mask_index}",
      f"<UNK>:{vectorizer.cbow_vocab.unk_index}")

# 打印样本向量化结果
sample_str = "Zhang"
vec = vectorizer.vectorize(sample_str)
print(f"样本'{sample_str}'的向量化结果:\n{vec}\n有效长度:{len(sample_str.split())}")

Char Vocab特殊标记索引： <MASK>:0 <UNK>:1
样本'Zhang'的向量化结果:
[148]
有效长度:1


## 模型结构验证

In [16]:
# 打印模型各层参数维度
print("\n模型结构：")
for name, param in classifier.named_parameters():
    print(f"{name.ljust(20)} | 维度：{tuple(param.size())}")


模型结构：
embedding.weight     | 维度：(6466, 50)
fc1.weight           | 维度：(6466, 50)
fc1.bias             | 维度：(6466,)


## RNN序列处理验证

In [33]:
# 初始化模型、优化器等
model = RNNClassifier(vocabulary_size=1000, embedding_size=128, hidden_size=256)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

# 创建一些虚拟数据
inputs = torch.randint(0, 1000, (32, 50))  # 32个样本，每个样本50个时间步
targets = torch.randint(0, 1000, (32,))

# 前向传播
outputs = model(inputs)
# 打印调试信息
print("Inputs shape:", inputs.shape)
print("Outputs shape:", outputs.shape)

# 计算损失
loss = criterion(outputs, targets)

# 反向传播和优化
optimizer.zero_grad()
loss.backward()
optimizer.step()
class RNNClassifier(nn.Module):
    def __init__(self, vocabulary_size, embedding_size, hidden_size, padding_idx=0):
        super(RNNClassifier, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocabulary_size,
                                      embedding_dim=embedding_size,
                                      padding_idx=padding_idx)
        self.rnn = nn.RNN(input_size=embedding_size,
                          hidden_size=hidden_size,
                          batch_first=True)
        self.fc = nn.Linear(hidden_size, vocabulary_size)

    def forward(self, x_in, x_lengths=None, apply_softmax=False):
        # 嵌入层
        x_embedded = self.embedding(x_in)
        # RNN层
        y_out, _ = self.rnn(x_embedded)
        # 聚合（取最后一个有效时间步的输出）
        if x_lengths is not None:
            # 动态索引最后一个有效时间步
            indices = x_lengths.view(-1, 1).expand(-1, y_out.size(2)) - 1
            y_out = y_out.gather(1, indices.unsqueeze(1)).squeeze(1)
        else:
            y_out = y_out[:, -1, :]
        # 全连接层
        y_out = self.fc(y_out)

        # 调试信息
        print(f"输入序列形状：{x_in.shape} -> 嵌入后：{x_embedded.shape}")
        print(f"RNN输出形状：{y_out.shape}（在聚合前为：{y_out.shape if x_lengths is None else f'{y_out.shape} -> 动态索引'}）")

        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)
        return y_out

输入序列形状：torch.Size([32, 50]) -> 嵌入后：torch.Size([32, 50, 128])
RNN输出形状：torch.Size([32, 1000])（在聚合前为：torch.Size([32, 1000])）
Inputs shape: torch.Size([32, 50])
Outputs shape: torch.Size([32, 1000])


## 模型性能评估

In [38]:
# 初始化模型、优化器和损失函数
model = RNNClassifier(vocabulary_size=len(vectorizer.cbow_vocab),
                      embedding_size=args.embedding_size,
                      hidden_size=128)  # RNN隐藏层大小
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
criterion = nn.CrossEntropyLoss()

# 训练模型
train_state = make_train_state(args)

# 训练循环
try:
    for epoch_index in range(args.num_epochs):
        print(f"\nEpoch [{epoch_index + 1}/{args.num_epochs}]")
        print("-" * 50)
        train_state['epoch_index'] = epoch_index

        # 训练集
        dataset.set_split('train')
        batch_generator = generate_batches(dataset,
                                          batch_size=args.batch_size,
                                          device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        model.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # 训练步骤
            optimizer.zero_grad()
            y_pred = model(batch_dict['x_data'])
            loss = criterion(y_pred, batch_dict['y_target'])
            loss.backward()
            optimizer.step()

            # 更新训练状态
            running_loss += (loss.item() - running_loss) / (batch_index + 1)
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            train_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # 验证集
        dataset.set_split('val')
        batch_generator = generate_batches(dataset,
                                          batch_size=args.batch_size,
                                          device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        model.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            y_pred = model(batch_dict['x_data'])
            loss = criterion(y_pred, batch_dict['y_target'])
            running_loss += (loss.item() - running_loss) / (batch_index + 1)
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            val_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=model, train_state=train_state)
        scheduler.step(train_state['val_loss'][-1])

        if train_state['stop_early']:
            break

except KeyboardInterrupt:
    print("Exiting loop")


split=train:   5%|██▏                                      | 13/240 [06:56<00:01, 170.77it/s, acc=0, epoch=0, loss=8.8]


Epoch [1/5]
--------------------------------------------------
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Si


split=train:  12%|████▎                               | 29/240 [06:57<00:01, 170.77it/s, acc=0.104, epoch=0, loss=8.79]

输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32


split=train:  18%|██████▍                            | 44/240 [06:57<00:01, 170.77it/s, acc=0.0694, epoch=0, loss=8.78]

...

split=train: 1198it [07:12, 82.51it/s, acc=27.7, epoch=4, loss=2.89]
split=train: 1199it [07:12, 82.51it/s, acc=27.8, epoch=4, loss=2.89]
split=val: 204it [07:12,  3.47it/s, acc=6.25, epoch=4, loss=2.93]
split=val: 205it [07:12,  3.69it/s, acc=6.25, epoch=4, loss=2.93]
split=val: 205it [07:12,  3.69it/s, acc=3.12, epoch=4, loss=2.83]
split=val: 206it [07:12,  3.69it/s, acc=3.12, epoch=4, loss=2.67]
split=val: 207it [07:12,  3.69it/s, acc=3.91, epoch=4, loss=2.62]
split=val: 208it [07:12,  3.69it/s, acc=3.12, epoch=4, loss=2.57]
split=val: 209it [07:12,  3.69it/s, acc=2.6, epoch=4, loss=2.55] 
split=val: 210it [07:12,  3.69it/s, acc=3.12, epoch=4, loss=2.56]
split=val: 211it [07:12,  3.69it/s, acc=2.73, epoch=4, loss=2.62]
split=val: 212it [07:12,  3.69it/s, acc=2.78, epoch=4, loss=2.63]
split=val: 213it [07:12,  3.69it/s, acc=2.5, epoch=4, loss=2.65] 
split=val: 214it [07:12,  3.69it/s, acc=2.56, epoch=4, loss=2.65]
split=val: 215it [07:12,  3.69it/s, acc=2.86, epoch=4, loss=2.61]
spli

输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32


split=val: 235it [07:12,  5.48it/s, acc=3.61, epoch=4, loss=2.63]
split=val: 236it [07:12,  5.48it/s, acc=3.6, epoch=4, loss=2.63] 
split=val: 237it [07:12,  5.48it/s, acc=3.58, epoch=4, loss=2.63]
split=val: 238it [07:12,  5.48it/s, acc=3.57, epoch=4, loss=2.63]
split=val: 239it [07:12,  7.96it/s, acc=3.57, epoch=4, loss=2.63]
split=val: 239it [07:12,  7.96it/s, acc=3.56, epoch=4, loss=2.63]
split=val: 240it [07:12,  7.96it/s, acc=3.55, epoch=4, loss=2.63]
split=val: 241it [07:12,  7.96it/s, acc=3.54, epoch=4, loss=2.63]
split=val: 242it [07:12,  7.96it/s, acc=3.53, epoch=4, loss=2.62]
split=val: 243it [07:12,  7.96it/s, acc=3.44, epoch=4, loss=2.62]
split=val: 244it [07:12,  7.96it/s, acc=3.43, epoch=4, loss=2.62]
split=val: 245it [07:12,  7.96it/s, acc=3.42, epoch=4, loss=2.62]
split=val: 246it [07:12,  7.96it/s, acc=3.49, epoch=4, loss=2.62]
split=val: 247it [07:12,  7.96it/s, acc=3.55, epoch=4, loss=2.63]
split=val: 248it [07:12,  7.96it/s, acc=3.47, epoch=4, loss=2.64]
split=val

输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32, 1, 50])
RNN输出形状：torch.Size([32, 6466])（在聚合前为：torch.Size([32, 6466])）
输入序列形状：torch.Size([32, 1]) -> 嵌入后：torch.Size([32


split=train: 1200it [07:22, 82.51it/s, acc=27.8, epoch=4, loss=2.89]
split=val: 255it [07:32,  7.96it/s, acc=3.62, epoch=4, loss=2.65]

## 推理能力验证

In [43]:
# 推理函数
def predict_nationality(surname, model, vectorizer, device, topk=5):
    """Predict the nationality of a surname

    Args:
        surname (str): the surname to predict
        model (RNNClassifier): the trained model
        vectorizer (CBOWVectorizer): the vectorizer used for preprocessing
        device (torch.device): the device on which the model is running
        topk (int, optional): the number of top predictions to return. Defaults to 5.

    Returns:
        list: a list of tuples containing the predicted nationalities and their probabilities
    """
    vectorized_surname = vectorizer.vectorize(surname)
    vectorized_surname = torch.tensor(vectorized_surname).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(vectorized_surname)
    
    probabilities = F.softmax(output, dim=1).squeeze()
    probabilities, indices = torch.topk(probabilities, topk)

    results = []
    for prob, idx in zip(probabilities, indices):
        nationality = vectorizer.cbow_vocab.lookup_index(idx.item())
        results.append((nationality, prob.item()))
    
    return results

# 预测指定姓氏的国籍
surnames_to_predict = ['McMahan', 'Nakamoto', 'Wan', 'Cho']

# 定义 device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载最佳模型
model_path = train_state['model_filename']
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()

for surname in surnames_to_predict:
    predictions = predict_nationality(surname, model, vectorizer, device)
    print(f"Surname: {surname}")
    for nationality, prob in predictions:
        print(f"  {nationality}: {prob:.4f}")
    print()

输入序列形状：torch.Size([1, 1]) -> 嵌入后：torch.Size([1, 1, 50])
RNN输出形状：torch.Size([1, 6466])（在聚合前为：torch.Size([1, 6466])）
Surname: McMahan
  Arabic: 0.1678
  Russian: 0.1552
  English: 0.1479
  Japanese: 0.0286
  German: 0.0282

输入序列形状：torch.Size([1, 1]) -> 嵌入后：torch.Size([1, 1, 50])
RNN输出形状：torch.Size([1, 6466])（在聚合前为：torch.Size([1, 6466])）
Surname: Nakamoto
  Arabic: 0.1678
  Russian: 0.1552
  English: 0.1479
  Japanese: 0.0286
  German: 0.0282

输入序列形状：torch.Size([1, 1]) -> 嵌入后：torch.Size([1, 1, 50])
RNN输出形状：torch.Size([1, 6466])（在聚合前为：torch.Size([1, 6466])）
Surname: Wan
  English: 0.1977
  Russian: 0.1475
  Arabic: 0.0990
  Italian: 0.0385
  Japanese: 0.0319

输入序列形状：torch.Size([1, 1]) -> 嵌入后：torch.Size([1, 1, 50])
RNN输出形状：torch.Size([1, 6466])（在聚合前为：torch.Size([1, 6466])）
Surname: Cho
  English: 0.1285
  Russian: 0.0986
  Arabic: 0.0436
  Japanese: 0.0367
  Italian: 0.0264

